# Run Crystal Explorer from QCrBox

Using the dotenv package makes things more convenient as we can read the environment variables from the .env.dev file in the QCrBox directory. If you want to run with python core packages only comment out the load_dotenv lines and insert the local and qcrbox pathes manually into the pathhelper from the .env.dev file.

In [1]:
import os
import shutil

from dotenv import load_dotenv

from qcrbox_wrapper import QCrBoxWrapper, QCrBoxPathHelper


if not load_dotenv('../.env.dev'):
    raise FileNotFoundError(
        ".dot.env file could not be loaded. Either adapt the path to your filesystem or "
        + "input the information loaded from os.environ manually"
    )

We create a path to run our Crystal Explorer example and get our paths from our own computer's filesystem and the internal filesystem of the QCrBox containers.

In [2]:
pathhelper = QCrBoxPathHelper(
    os.environ['QCRBOX_SHARED_FILES_DIR_HOST_PATH'],
    os.environ['QCRBOX_SHARED_FILES_DIR_CONTAINER_PATH'],
    'examples_crystal_explorer'
)

path_local = pathhelper.local_path
path_qcrbox = pathhelper.qcrbox_path

## Connecting to QCrBox

We can connect to the QCrBox Inventory via python after we have started everything with `qcb up`. Sometimes the server takes a while so you might need to retry if it initially refuses connection. This should not take more than 30 seconds after your console output says that everything has started.

In [3]:
gui_infos = {
    'CrystalExplorer': {'port': os.environ['QCRBOX_CRYSTAL_EXPLORER_PORT'], 'commands': ['interactive']}
}

qcrbox = QCrBoxWrapper('127.0.0.1', 11000, gui_infos)

We should see `Crystalexplorer in our application_dict and should put it into a variable.

In [4]:
qcrbox.application_dict


{'CrystalExplorer': CrystalExplorer()}

In [5]:
crystal_explorer = qcrbox.application_dict['CrystalExplorer']


## Running the Crystal Explorer GUI in QCrBox

We can now access the GUI of Crystal Explorer within the container. To run the command needs a unified cif, which is then internally converted into the format CrystalExplorer needs. For more read the Olex2 Example Notebook (TODO Write and link a more complete explanation in docs). 

Once we run the `interactive` command, two things happen: Firstly, a new browser window with the GUI opens up. Secondly, you should see an input prompt in your execution engine for this notebook. (In Jupyter this should be at the end of the cell, in VSCode it is at the top). By pressing enter within this prompt we tell QCrBox that we are done. There is no back conversion, as we do not expect Crystal Explorer to modify the cif file. Note that this input prompt is a stopgap for an actual UI, so this feels a bit clunky at the moment.

In [6]:
shutil.copy('./input_files/input.cif', path_local / 'input.cif')

calc = crystal_explorer.interactive(
    input_cif_path=path_qcrbox / 'input.cif'
)
calc.status

QCrBoxCalculationStatus(calculation_id=2, command_id=1, started_at='2024-03-01T11:39:26.374470', status='running', status_details={'status': 'running', 'details': {'returncode': None}})